In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import sklearn
import seaborn as sns
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cols =["duration",
"protocol_type",
"service",
"flag",
"src_bytes",
"dst_bytes",
"land",
"wrong_fragment",
"urgent",
"hot",
"num_failed_logins",
"logged_in",
"num_compromised",
"root_shell",
"su_attempted",
"num_root",
"num_file_creations",
"num_shells",
"num_access_files",
"num_outbound_cmds",
"is_host_login",
"is_guest_login",
"count",
"srv_count",
"serror_rate",
"srv_serror_rate",
"rerror_rate",
"srv_rerror_rate",
"same_srv_rate",
"diff_srv_rate",
"srv_diff_host_rate",
"dst_host_count",
"dst_host_srv_count",
"dst_host_same_srv_rate",
"dst_host_diff_srv_rate",
"dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate",
"dst_host_serror_rate",
"dst_host_srv_serror_rate",
"dst_host_rerror_rate",
"dst_host_srv_rerror_rate",
"attack"]

attacks_types = {
    'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'dos':'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
'r2l':'r2l',
'u2r':'u2r',
'probe':'probe'
}

In [ ]:
df = pd.read_csv('/kaggle/input/kdddata/Dataset.csv')
df.columns = cols

df['attack'] = df['attack'].apply(lambda r:attacks_types[r])

df.head()

In [ ]:
# Finding categorical features
num_cols = df._get_numeric_data().columns
  
cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('attack')
  
cate_cols

In [ ]:
df = df.dropna('columns')# drop columns with NaN
  
df = df[[col for col in df if df[col].nunique() > 1]]# keep columns where there are more than 1 unique values

corr = df.corr()
  
plt.figure(figsize =(15, 12))
  
sns.heatmap(corr)
  
plt.show()

In [ ]:
# This variable is highly correlated with num_compromised and should be ignored for analysis.
#(Correlation = 0.9938277978738366)
df.drop('num_root', axis = 1, inplace = True)
  
# This variable is highly correlated with serror_rate and should be ignored for analysis.
#(Correlation = 0.9983615072725952)
df.drop('srv_serror_rate', axis = 1, inplace = True)
  
# This variable is highly correlated with rerror_rate and should be ignored for analysis.
#(Correlation = 0.9947309539817937)
df.drop('srv_rerror_rate', axis = 1, inplace = True)
  
# This variable is highly correlated with srv_serror_rate and should be ignored for analysis.
#(Correlation = 0.9993041091850098)
df.drop('dst_host_srv_serror_rate', axis = 1, inplace = True)
  
# This variable is highly correlated with rerror_rate and should be ignored for analysis.
#(Correlation = 0.9869947924956001)
df.drop('dst_host_serror_rate', axis = 1, inplace = True)
  
# This variable is highly correlated with srv_rerror_rate and should be ignored for analysis.
#(Correlation = 0.9821663427308375)
df.drop('dst_host_rerror_rate', axis = 1, inplace = True)
  
# This variable is highly correlated with rerror_rate and should be ignored for analysis.
#(Correlation = 0.9851995540751249)
df.drop('dst_host_srv_rerror_rate', axis = 1, inplace = True)
  
# This variable is highly correlated with srv_rerror_rate and should be ignored for analysis.
#(Correlation = 0.9865705438845669)
df.drop('dst_host_same_srv_rate', axis = 1, inplace = True)

In [ ]:
# protocol_type feature mapping
pmap = {'icmp':0, 'tcp':1, 'udp':2}
df['protocol_type'] = df['protocol_type'].map(pmap)

In [ ]:
# flag feature mapping
fmap = {'SF':0, 'S0':1, 'REJ':2, 'RSTR':3, 'RSTO':4, 'SH':5, 'S1':6, 'S2':7, 'RSTOS0':8, 'S3':9, 'OTH':10}
df['flag'] = df['flag'].map(fmap)

In [ ]:
df.drop('service', axis = 1, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Target variable and train set
y = df[['attack']]
X = df.drop(['attack', ], axis = 1)
  
sc = MinMaxScaler()
X = sc.fit_transform(X)
  
# Split test and train data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
# data visualization
plt.figure(figsize=(30,20))

plt.title("Types of attack in dataset")

sns.barplot(x="attack", y=y.index, data=y)

In [ ]:
X_rfetrain = X_train

In [ ]:
print(X_rfetrain.shape)

In [ ]:
X_test2=X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rfe=RandomForestClassifier(max_depth=6,n_estimators=10,n_jobs=2)
clf_rfe.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_rfe.predict(X_test2)
# Create confusion matrix
#pd.crosstab(Y_test, Y_pred2, rownames=['Actual attacks'], colnames=['Predicted attacks'])
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
ac_rfe = accuracy_score(y_test,Y_pred2)
ac_rfe

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf_KNN=KNeighborsClassifier(n_neighbors=1000)
clf_KNN.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_KNN.predict(X_test2)
# Create confusion matrix
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
# accuracy = cross_val_score(clf_KNN, X_test2, y_test, cv=10, scoring='accuracy')
# print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))
ac_knn = accuracy_score(y_test,Y_pred2)
ac_knn

In [ ]:
from sklearn.svm import SVC

clf_SVM=SVC(kernel='linear', C=1.0, random_state=0)
clf_SVM.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_SVM.predict(X_test2)
# Create confusion matrix
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
ac_svc = accuracy_score(y_test,Y_pred2)
ac_svc

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lg = LogisticRegression(random_state=0,solver='liblinear',C=10.0)
clf_lg.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_lg.predict(X_test2)
# Create confusion matrix
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
ac_lr = accuracy_score(y_test, Y_pred2)
ac_lr

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(max_depth = 2,random_state=0)

clf_ab = AdaBoostClassifier(base_estimator=clf1,n_estimators=1,random_state=0)
clf_ab.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_ab.predict(X_test2)
# Create confusion matrix
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
ac_dead = accuracy_score(y_test, Y_pred2)
ac_dead

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dt=DecisionTreeClassifier(max_depth = 4,random_state=0)
clf_dt.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_dt.predict(X_test2)
# Create confusion matrix
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
ac_de = accuracy_score(y_test, Y_pred2)
ac_de

In [ ]:
from sklearn.ensemble import VotingClassifier

clf_voting = VotingClassifier(estimators=[('rf', clf_rfe), ('knn', clf_KNN), ('svm', clf_SVM), ('lR', clf_lg), ('Adabst', clf_ab), ('dt', clf_dt)], voting='hard')
clf_voting.fit(X_rfetrain, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred2=clf_voting.predict(X_test2)
# Create confusion matrix
cf_matrix = confusion_matrix(y_test,Y_pred2)
plt.figure(figsize=(30,20))

sns.heatmap(cf_matrix, annot=True)

In [ ]:
ac_vt = accuracy_score(y_test, Y_pred2)
ac_vt

In [ ]:
plt.figure(figsize=(15,5))

plt.title("Classifiers results")

plt.xlabel("Classifiers")

plt.ylabel("accuracy")

acs = {
    "Classifier":["Random Forest","KNN","SVC","Logistic Regression","Decision Tree with Adaboost","Decision Tree","Voting Classifier"],
    "Accuracy":[ac_rfe*100,ac_knn*100,ac_svc*100,ac_lr*100,ac_dead*100,ac_de*100,ac_vt*100]
}

df = pd.DataFrame(data=acs)

ax = sns.barplot(x="Classifier", y="Accuracy", data=df)

ax

for i in ax.containers:
    ax.bar_label(i,)

